In [21]:
from pathlib import Path
import json
import pickle
import keras  # <- pakai keras standalone, bukan tf.keras

# 1) Tentukan path bundle model
ROOT = Path().resolve()                      # .../smartphone-battery-be
BASE = ROOT / "models" / "latest"

print("BASE:", BASE)

# 2) Load model
model = keras.models.load_model(
    BASE / "model.keras",
    compile=False,      # kita hanya mau inference
    safe_mode=False     # izinkan layer/ops yang tidak diregister (mis. NotEqual)
)
print("[OK] model loaded")

# 3) Load scaler & config
with open(BASE / "scaler.pkl", "rb") as f:
    scaler = pickle.load(f)

with open(BASE / "config.json") as f:
    config = json.load(f)

WINDOW_SIZE = config["window_size"]
FEATURE_COLS = config["feature_cols"]

print("window_size:", WINDOW_SIZE)
print("feature_cols:", FEATURE_COLS)


BASE: C:\College\Bachelor\smartphone-battery-be\notebooks\models\latest
[OK] model loaded
window_size: 24
feature_cols: ['batt_voltage_v', 'batt_temp_c', 'throughput_total_mbps', 'energy_per_bit_avg_J', 'EFC', 'soh_trend', 'efc_delta', 'temp_ema', 'temp_max_win', 'tp_ema', 'epb_ema', 'batt_voltage_v_z', 'batt_temp_c_z', 'throughput_total_mbps_z', 'energy_per_bit_avg_J_z', 'SoH_filled_z', 'EFC_z', 'soh_trend_z', 'BoT_mAh_per_Gbps']


In [22]:
import sys
from pathlib import Path

CWD = Path().resolve()

# Cari folder yang punya 'src'
if (CWD / "src").exists():
    ROOT = CWD
elif (CWD.parent / "src").exists():
    ROOT = CWD.parent
elif (CWD.parent.parent / "src").exists():
    ROOT = CWD.parent.parent
else:
    raise RuntimeError(f"Tidak menemukan folder 'src' dari CWD={CWD}")

if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

print("ROOT set to:", ROOT)
print("ROOT/src exists?:", (ROOT / "src").exists())

ROOT set to: C:\College\Bachelor\smartphone-battery-be
ROOT/src exists?: True


In [ ]:
import pandas as pd
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
%matplotlib inline

from src.core.aging_features import add_aging_features
from src.core.feature_engineering import add_per_device_zscore
from src.core.sequence import create_sequences
from src.utils.utils import connect_to_db
from src.utils.utils import calculate_soh_and_cycles
from src.core.throughput_energy import (
  calculate_energy_consumption, 
  calculate_throughput, 
  calculate_throughput_energy_and_bot
)

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

# Define database connection parameters
PORT = os.getenv("PORT")
USERNAME = os.getenv("USERNAME")
PASSWORD = os.getenv("PASSWORD")
HOST = os.getenv("HOST")
DATABASE = os.getenv("DATABASE")

if not all([PORT, USERNAME, PASSWORD, HOST, DATABASE]):
  print("Error: One or more database connection parameters are missing.")
  sys.exit(1)

In [24]:
# DB connection
conn = connect_to_db(HOST, PORT, DATABASE, USERNAME, PASSWORD)

# Load raw data
with conn.cursor() as cursor:
  cursor.execute("""
    SELECT * 
    FROM public.raw_metrics 
    WHERE device_id = '2311DRK48G-b135dcd1d7e9320f'
  """)
  data = cursor.fetchall()
  df = pd.DataFrame(data, columns=[desc[0] for desc in cursor.description])

# Preprocess timestamp & sort
df["created_at"] = pd.to_datetime(df["created_at"])
df = df.sort_values(["device_id", "created_at"]).reset_index(drop=True)

Connection to database established successfully.


In [ ]:
# Calculate SoH and charge cycles
df_1 = calculate_throughput(df)
df_2 = calculate_energy_consumption(df_1)
df_3 = calculate_throughput_energy_and_bot(df_2)

In [26]:
# Merge SoH calculation for all devices
df_all = (
  df_3
  .groupby("device_id", group_keys=False)
  .apply(lambda g: calculate_soh_and_cycles(g))
)

df_all["SoH_filled"] = df_all["SoH_smooth"].where(
  df_all["SoH_smooth"].notna(), df_all["SoH"]
)

df_all["SoH_filled"] = (
  df_all.groupby("device_id")["SoH_filled"]
    .transform(lambda s: s.ffill().bfill())
)
df_feat = add_aging_features(df_all)
df_feat_z = add_per_device_zscore(df_feat)

C:\Users\Fadel\AppData\Local\Temp\ipykernel_32576\2536902333.py:5: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: calculate_soh_and_cycles(g))
C:\College\Bachelor\smartphone-battery-be\src\core\aging_features.py:57: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(DEVICE_COL).apply(_per_dev).reset_index(drop=True)
C:\College\Bachelor\smartphone-battery-be\src\core\feature_engineering.py:9

In [ ]:
# Check columns for debugging
print(sorted(df.columns))
print(sorted(df_feat_z.columns))

['batt_current_ua', 'batt_temp_c', 'batt_voltage_mv', 'battery_capacity_pct', 'battery_health', 'battery_level', 'channel_quality', 'charge_counter_uah', 'charge_source', 'created_at', 'current_avg_ua', 'cycles_count', 'device_id', 'energy_nwh', 'fg_pkg', 'id', 'is_charging', 'net_type', 'rx_total_bytes', 'ts_utc', 'tx_total_bytes', 'user_id']
['BoT_mAh_per_Gbps', 'Ct_mAh', 'EFC', 'EFC_z', 'Q_mAh', 'SoH', 'SoH_filled', 'SoH_filled_z', 'SoH_pct', 'SoH_smooth', 'SoH_smooth_pct', 'batt_temp_c', 'batt_temp_c_z', 'batt_voltage_v', 'batt_voltage_v_z', 'created_at', 'delta_Q_mAh', 'delta_t_s', 'device_id', 'discharge_mAh', 'efc_delta', 'efc_delta_z', 'energy_per_bit_avg_J', 'energy_per_bit_avg_J_z', 'energy_per_bit_rx', 'energy_per_bit_rx_J', 'energy_per_bit_tx', 'energy_per_bit_tx_J', 'energy_wh', 'epb_ema', 'epb_ema_z', 'soh_ema_fast', 'soh_ema_slow', 'soh_trend', 'soh_trend_z', 'temp_ema', 'temp_ema_z', 'temp_max_win', 'temp_max_win_z', 'throughput_total_bps', 'throughput_total_mbps', 'thr

In [ ]:
# Build sequences
X_all, y_all = create_sequences(df_feat_z, feature_cols=FEATURE_COLS, window=WINDOW_SIZE)

# Last window for inference
X = X_all[-1:]

# Scaling
X_flat = X.reshape(-1, X.shape[-1])        
X_scaled_flat = scaler.transform(X_flat)
X_scaled = X_scaled_flat.reshape(1, WINDOW_SIZE, len(FEATURE_COLS))

# Predict SoH
y_pred = model.predict(X_scaled)[0]
soh_pred = round(float(y_pred) * 100.0, 1)

# SoH "ground truth"
soh_now = float(df_feat["SoH_filled"].iloc[-1])
print("SoH_now :", soh_now)
print("SoH_pred:", soh_pred)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
SoH_now : nan
SoH_pred: 50.8


c:\College\Bachelor\smartphone-battery-be\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Fadel\AppData\Local\Temp\ipykernel_32576\1473017637.py:21: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  soh_pred = round(float(y_pred) * 100.0, 1)


In [30]:
print("Model input_shape:", model.input_shape)
print("WINDOW_SIZE (config):", WINDOW_SIZE)
print("len(FEATURE_COLS):", len(FEATURE_COLS))

Model input_shape: (None, 24, 19)
WINDOW_SIZE (config): 24
len(FEATURE_COLS): 19
